<a href="https://colab.research.google.com/github/TanishDevX/License-Plate-Recognition-/blob/main/Copy_of_ALPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔧 STEP 1: INSTALL DEPENDENCIES
# -------------------------------
!pip install ultralytics --upgrade
!pip install paddleocr
!apt-get install -y libgl1-mesa-glx

print("✅ YOLOv11 & PaddleOCR Installed!")


In [ ]:
# ============================================================
# 📌 STEP 2: SET DATASET PATH IN DRIVE


DATASET_PATH = "/content/drive/MyDrive/Dataset"

import os
if not os.path.exists(DATASET_PATH):
    raise Exception("❌ Dataset path incorrect. Check your folder path in Drive.")

print("📁 Dataset found at:", DATASET_PATH)


In [ ]:
# ============================================================
# 🔍 STEP 3: Locate data.yaml inside dataset
# ============================================================
yaml_path = None

for root, dirs, files in os.walk(DATASET_PATH):
    if "data.yaml" in files:
        yaml_path = os.path.join(root, "data.yaml")
        break

if yaml_path is None:
    raise Exception("❌ data.yaml NOT FOUND! Make sure dataset is YOLOv11 formatted.")

print("📌 YAML Found at:", yaml_path)

In [ ]:
from ultralytics import YOLO

print("🔥 Loading YOLOv11n...")
model = YOLO("yolo11n.pt")

# -------------------------
# STAGE 4 - FAST TRAIN
# -------------------------
results1 = model.train(
    data=yaml_path,
    epochs=6,
    imgsz=640,
    batch=4,
    freeze=10,
    device=0,
    cache=False,
    amp=True
)


model = YOLO("runs/detect/train/weights/last.pt")

# -------------------------
# STAGE 5 - FINE TUNE
# -------------------------
results2 = model.train(
    data=yaml_path,
    epochs=4,
    imgsz=640,
    batch=2,
    freeze=0,
    device=0,
    cache=False,
    amp=True
)

print("🎉 Training complete ")


In [ ]:
!pip install paddlepaddle==3.2.2
!pip install paddleocr



In [ ]:
# ============================================================
# 🔤 STEP 6: Load PaddleOCR
# ============================================================
from paddleocr import PaddleOCR
ocr = PaddleOCR(lang="en", use_angle_cls=True)
print("🔤 OCR Loaded!")

In [ ]:
# ============================================
# 🚗 STEP 7: DETECT → CROP → OCR FUNCTION
# ============================================

import cv2
import matplotlib.pyplot as plt

# Load your trained YOLO model
model = YOLO("/content/best.pt")   # use best.pt for inference

def detect_and_read_plate(img_path):
    print(f"📸 Processing: {img_path}")

    img = cv2.imread(img_path)
    if img is None:
        print("❌ Error: Image not found")
        return ""

    # -----------------------------
    # 1) YOLO DETECTION
    # -----------------------------
    results = model(img, verbose=False)

    if results[0].boxes is None or len(results[0].boxes.xyxy) == 0:
        print("❌ No license plate detected!")
        return ""

    x1, y1, x2, y2 = map(int, results[0].boxes.xyxy[0])
    crop = img[y1:y2, x1:x2]

    # show crop
    plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
    plt.title("Detected Plate")
    plt.axis("off")
    plt.show()

    # -----------------------------
    # 2) PREPROCESS
    # -----------------------------
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    upscale = cv2.resize(gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)

    # Convert grayscale back to 3-channel BGR for PaddleOCR
    upscale_color = cv2.cvtColor(upscale, cv2.COLOR_GRAY2BGR)

    # -----------------------------
    # 3) OCR USING PADDLEOCR (predict)
    # -----------------------------
    ocr_result = ocr.predict(upscale_color)

    text = ""
    try:
        for det in ocr_result[0]['rec_texts']:
            text += det + " "
    except:
        pass

    text = text.strip()

    print("🔤 OCR Result:", text if text else "❌ No text detected")
    return text



In [ ]:
detect_and_read_plate("/content/drive/MyDrive/Plates/img6.jpg")



In [ ]:
# ============================================================
# 📊 CHECK YOLOv11 ACCURACY (MAP, PRECISION, RECALL)
# ============================================================
results = model.val()

print("\n📌 VALIDATION METRICS:")
print(f"mAP50:       {results.box.map50:.4f}")
print(f"mAP50-95:    {results.box.map:.4f}")
print(f"Precision:   {results.box.mp:.4f}")
print(f"Recall:      {results.box.mr:.4f}")
print("\n✅ Accuracy Check Completed!")


In [ ]:
from google.colab import files
files.download('runs/detect/train/weights/best.pt')


In [ ]:
from google.colab import files
uploaded = files.upload()  # upload best.pt


In [ ]:
from ultralytics import YOLO
model = YOLO("best.pt")
